<a href="https://colab.research.google.com/github/NiharikaShrivastava/Socila-Media-Recommender-System/blob/main/Social_Media_Content_Recommender_System_(Machine_Learning_ll).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pandas as pd
from datetime import datetime, timedelta

class SocialMediaRecommender:
    def __init__(self):
        self.users = {}  # User profiles and preferences
        self.content = {}  # Content items and their features
        self.interactions = defaultdict(dict)  # User-content interactions
        self.user_similarity_matrix = None
        self.content_similarity_matrix = None

    def add_user(self, user_id, features):
        """
        Add a user with their profile features (interests, demographics, etc.)
        """
        self.users[user_id] = {
            'features': features,
            'timestamp': datetime.now()
        }

    def add_content(self, content_id, features, content_type, author_id):
        """
        Add content item with its features, type, and author
        """
        self.content[content_id] = {
            'features': features,
            'type': content_type,
            'author_id': author_id,
            'timestamp': datetime.now(),
            'engagement_score': 0
        }

    def add_interaction(self, user_id, content_id, interaction_type, timestamp):
        """
        Record user interaction with content (view, like, share, etc.)
        """
        self.interactions[user_id][content_id] = {
            'type': interaction_type,
            'timestamp': timestamp
        }

        # Update content engagement score
        interaction_weights = {
            'view': 1,
            'like': 3,
            'share': 5,
            'comment': 4
        }
        self.content[content_id]['engagement_score'] += interaction_weights.get(interaction_type, 1)

    def _update_similarity_matrices(self):
        """
        Update user and content similarity matrices
        """
        # Create user feature matrix
        user_features = np.array([user['features'] for user in self.users.values()])
        self.user_similarity_matrix = cosine_similarity(user_features)

        # Create content feature matrix
        content_features = np.array([item['features'] for item in self.content.values()])
        self.content_similarity_matrix = cosine_similarity(content_features)

    def _get_user_preferences(self, user_id):
        """
        Calculate user preferences based on their interactions
        """
        preferences = defaultdict(float)
        if user_id in self.interactions:
            for content_id, interaction in self.interactions[user_id].items():
                if content_id in self.content:
                    content_type = self.content[content_id]['type']
                    preferences[content_type] += 1
        return preferences

    def recommend_content(self, user_id, n_recommendations=5):
        """
        Generate content recommendations for a user
        """
        if user_id not in self.users:
            return []

        self._update_similarity_matrices()

        # Get user's recent interactions
        user_recent_interactions = set()
        if user_id in self.interactions:
            user_recent_interactions = set(self.interactions[user_id].keys())

        # Calculate recommendations using multiple signals
        recommendations = []
        user_preferences = self._get_user_preferences(user_id)

        for content_id, content in self.content.items():
            if content_id in user_recent_interactions:
                continue

            score = 0

            # Content-based similarity
            user_features = self.users[user_id]['features']
            content_features = content['features']
            content_similarity = cosine_similarity([user_features], [content_features])[0][0]
            score += content_similarity * 0.4

            # Collaborative filtering signal
            similar_users = self._get_similar_users(user_id, n=5)
            cf_score = self._calculate_cf_score(content_id, similar_users)
            score += cf_score * 0.3

            # Engagement signal
            engagement_score = content['engagement_score']
            score += (engagement_score / 100) * 0.2

            # Freshness signal
            days_old = (datetime.now() - content['timestamp']).days
            freshness_score = 1 / (1 + days_old)
            score += freshness_score * 0.1

            recommendations.append((content_id, score))

        # Sort and filter recommendations
        recommendations.sort(key=lambda x: x[1], reverse=True)
        return recommendations[:n_recommendations]

    def _get_similar_users(self, user_id, n=5):
        """
        Find similar users based on user similarity matrix
        """
        if self.user_similarity_matrix is None:
            return []

        user_idx = list(self.users.keys()).index(user_id)
        similar_user_indices = np.argsort(self.user_similarity_matrix[user_idx])[-n-1:-1]
        return [list(self.users.keys())[idx] for idx in similar_user_indices]

    def _calculate_cf_score(self, content_id, similar_users):
        """
        Calculate collaborative filtering score based on similar users' interactions
        """
        cf_score = 0
        for similar_user in similar_users:
            if similar_user in self.interactions and content_id in self.interactions[similar_user]:
                interaction_type = self.interactions[similar_user][content_id]['type']
                if interaction_type == 'like':
                    cf_score += 2
                elif interaction_type == 'share':
                    cf_score += 3
                else:
                    cf_score += 1
        return cf_score / (len(similar_users) + 1)

# Example usage
def demo_recommender():
    recommender = SocialMediaRecommender()

    # Add users with feature vectors (interests, demographics, etc.)
    recommender.add_user('user1', [1, 0, 1, 0, 1])  # Tech, Sports, Travel
    recommender.add_user('user2', [0, 1, 1, 1, 0])  # Fashion, Travel, Food

    # Add content items with feature vectors
    recommender.add_content('post1', [1, 0, 0, 0, 1], 'tech_article', 'author1')
    recommender.add_content('post2', [0, 1, 1, 0, 0], 'travel_post', 'author2')

    # Add some interactions
    recommender.add_interaction('user1', 'post1', 'like', datetime.now())
    recommender.add_interaction('user2', 'post2', 'share', datetime.now())

    # Get recommendations for user1
    recommendations = recommender.recommend_content('user1', n_recommendations=2)

    # Get recommendations for user2
    recommendations = recommender.recommend_content('user2', n_recommendations=2)

    return recommendations

if __name__ == "__main__":
    recommendations = demo_recommender()
    print("Recommended content:", recommendations)

Recommended content: [('post1', np.float64(0.406))]
